In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# classificador
from sklearn.naive_bayes import GaussianNB

# normalização dos dados
from sklearn.preprocessing import Normalizer

# algoritmo de agrupamento
from sklearn.cluster import KMeans

# Validação do agrupamento
from sklearn.metrics import adjusted_rand_score

# abordagem de divisão de conjunto de treino e teste.
from sklearn.model_selection import train_test_split


database = pd.read_csv("data/dados.csv")
database.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 20 columns):
age               2999 non-null int64
job               2999 non-null object
marital           2999 non-null object
education         2999 non-null object
default           2999 non-null object
housing           2999 non-null object
loan              2999 non-null object
contact           2999 non-null object
month             2999 non-null object
day_of_week       2999 non-null object
duration          2999 non-null int64
campaign          2999 non-null int64
pdays             2999 non-null int64
previous          2999 non-null int64
poutcome          2999 non-null object
emp.var.rate      2999 non-null float64
cons.price.idx    2999 non-null float64
cons.conf.idx     2999 non-null float64
euribor3m         2999 non-null float64
nr.employed       2999 non-null float64
dtypes: float64(5), int64(5), object(10)
memory usage: 468.7+ KB


In [2]:
database.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,2999.000000,2999.000000,2999.000000,2999.000000,2999.000000,2999.000000,2999.000000,2999.000000,2999.000000,2999.000000
mean,39.889296,255.085028,2.526842,962.227743,0.192064,0.079093,93.584553,-40.578226,3.605769,5165.642314
std,10.282229,256.265737,2.556995,187.599866,0.551960,1.559126,0.581162,4.594237,1.739625,74.384632
min,18.000000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.635000,4963.600000
25%,32.000000,101.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.334000,5099.100000
50%,38.000000,180.000000,2.000000,999.000000,0.000000,1.100000,93.876000,-41.800000,4.857000,5191.000000
75%,47.000000,316.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,88.000000,3643.000000,35.000000,999.000000,6.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


In [3]:
obj_df = database.select_dtypes(include=['object']).copy()
obj_df.head()

,job,marital,education,default,housing,loan,contact,month,day_of_week,poutcome
0,blue-collar,married,basic.9y,no,yes,no,cellular,may,fri,nonexistent
1,services,single,high.school,no,no,no,telephone,may,fri,nonexistent
2,services,married,high.school,no,yes,no,telephone,jun,wed,nonexistent
3,services,married,basic.9y,no,unknown,unknown,telephone,jun,fri,nonexistent
4,admin.,married,university.degree,no,yes,no,cellular,nov,mon,nonexistent


In [4]:
#Dicionário de dados
list = { "job" :{"unknown":1,"admin.":1,"blue-collar":2,"technician":3,"services":4,"management":5,"retired":6,"self-employed":7,"entrepreneur":8,"unemployed":9,"housemaid":10,"student":11},
         "marital" :{"unknown":1,"married":1, "single":2, "divorced":3},
         "education": {"unknown":6,"basic.4y":1,"basic.6y":2,"basic.9y":3,"high.school":4,"professional.course":5,"university.degree":6},
         "default": {},
         "housing": {"unknown":1,"no":0,"yes":1},
         "loan": {"unknown":0,"no":0,"yes":1},
         "contact": {"cellular":0,"telephone":1},
         "month": {"jan":1,"feb":2,"mar":3,"apr":4,"may":5,"jun":6,"jul":7,"aug":8,"sep":9,"oct":10,"nov":11,"dec":12},
         "day_of_week": {"thu":1,"tue":2,"mon":3,"wed":4,"fri":5},
         "poutcome": {"nonexistent":1,"failure":2,"success":3}
        }

# Substituindo 
database.replace(list, inplace=True)

In [5]:
del database['default']
del database['campaign']
del database['pdays']

In [6]:
display(database)

,age,job,marital,education,housing,loan,contact,month,day_of_week,duration,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,30,2,1,3,1,0,0,5,5,487,0,1,-1.8,92.893,-46.2,1.313,5099.1
1,39,4,2,4,0,0,1,5,5,346,0,1,1.1,93.994,-36.4,4.855,5191.0
2,25,4,1,4,1,0,1,6,4,227,0,1,1.4,94.465,-41.8,4.962,5228.1
3,38,4,1,3,1,0,1,6,5,17,0,1,1.4,94.465,-41.8,4.959,5228.1
4,47,1,1,6,1,0,0,11,3,58,0,1,-0.1,93.200,-42.0,4.191,5195.8
5,32,4,2,6,0,0,0,9,1,128,2,2,-1.1,94.199,-37.5,0.884,4963.6
6,32,1,2,6,1,0,0,9,3,290,0,1,-1.1,94.199,-37.5,0.879,4963.6
7,41,8,1,6,1,0,0,11,3,44,0,1,-0.1,93.200,-42.0,4.191,5195.8
8,31,4,3,5,0,0,0,11,2,68,1,2,-0.1,93.200,-42.0,4.153,5195.8
9,35,2,1,3,0,0,1,5,1,170,0,1,1.1,93.994,-36.4,4.855,5191.0


In [7]:
classes = database.poutcome
del_columns = ['duration']
database = database.drop(del_columns, axis=1)
print(database.columns)

Index(['age', 'job', 'marital', 'education', 'housing', 'loan', 'contact',
       'month', 'day_of_week', 'previous', 'poutcome', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed'],
      dtype='object')


In [8]:
#dividindo em base de treino e teste
X_train, X_test, y_train, y_test = train_test_split(database, classes, random_state=0)

In [9]:
# normaliza os dados de treino
scaler = Normalizer().fit(X_train)
database = scaler.transform(X_train)

# agrupamento  de dados pelo KMeans
kmeans = KMeans(n_clusters=2, random_state=0)

In [10]:
# Agrupamento dos objetos contidos no conjunto de treino (pode ser o agrupamento do conjunto todo também).
model = kmeans.fit(X_train)

In [11]:
# os rótulos obtidos
print(model.labels_)
print(len(model.labels_))

[0 0 1 ... 1 0 0]
2249


In [12]:
# validação do agrupamento feito pelo KMeans (usando o rand index ajustado)
# compara os rótulos originais dos objetos de treino com os rótulos de grupo formados pelo KMeans
print("Score: ", adjusted_rand_score(y_train, model.labels_))

Score:  0.2488430468697579


In [13]:
previsao = pd.DataFrame()
previsao["Predicted"] = model.labels_

previsao.to_csv('prediction.csv',index=True,index_label="Id")

In [14]:
new = pd.read_csv('prediction.csv')
display(new)
display(new.shape)

,Id,Predicted
0,0,0
1,1,0
2,2,1
3,3,1
4,4,1
5,5,0
6,6,1
7,7,0
8,8,0
9,9,0


(2249, 2)